# Optimization Methods for Data Science
### A.A. 2024-2025

Alessandro Pannone
Ph.D. Student @ Sapienza University of Rome

pannone@diag.uniroma1.it

## Matplotlib
Documentation: https://matplotlib.org/

Matplotlib is a Python library for 2D and 3D plotting in Python.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
# DRAW SIMPLE PLOT
x = np.linspace(0, 2, 100)
y = np.random.random(100)

plt.plot(x,y,'g-')
plt.xlabel('x label')
plt.ylabel('y label')
plt.title("Simple Plot")
plt.show()

And what if I want to compare different __2D functions__ ?

In [ ]:
plt.plot(x, x, color='red')
plt.plot(x, x**2, color='blue')
plt.plot(x, 2**x, color='green')

plt.xlabel('x label')
plt.ylabel('y label')

plt.title("Comparing functions")

plt.legend(['Linear','Quadratic','Exponential'])

plt.show()

And what about __plotting a 3D function__?

In [ ]:
from typing import Callable

def plotting(funct: Callable, title: str):
    #create the object empty figure
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    #create the grid
    x = np.linspace(-5, 5, 50) #create 50 points between [-5,5] evenly spaced
    y = np.linspace(-5, 5, 50)
    X, Y = np.meshgrid(x, y) #create the grid for the plot

    Z = funct(X, Y) #evaluate the function (note that X,Y,Z are matrix)


    ax.plot_surface(X, Y, Z, rstride=1, cstride=1,cmap='viridis', edgecolor='none')

    ax.set_xlabel('x')
    ax.set_ylabel('y')
    ax.set_zlabel('z')
    ax.set_title(title)
    plt.show()

In [ ]:
def paraboloid(x,y):
    return y**2 + x**2

plotting(paraboloid,'A simple paraboloid')

## Unconstrained Optimization in Python

Suppose you have a function $f: \mathbb{R}^n \rightarrow \mathbb{R}$ you want to optimize, i.e. you are solving:
$$ \min_{x \in \mathbb{R}^n} f(x)
$$

If you are in a supervised learning contest, $f(x)$ could be your error function depending on the parameters of your neural network.

Let's think your objective function is:
$$ f(x) = 4x_{1}^2+10x_{2}^2+12x_{1}^2 x_{2}^2
$$
where $x = (x_1,x_2) \in \mathbb{R}^2$

We firstly write the function and its gradient.

In [ ]:
# DEFINE THE FUNCTION TO OPTIMIZE
def f(x: np.ndarray) -> np.ndarray:
    """
    NOTE: If you want to optimize a function, it should take only ONE VARIABLE argument --> put them inside an array!
    NOTE: Variable argument must be the FIRST ARGUMENT of the function! (then, you can add as many arguments you want)
    """

    x1 = x[0]
    x2 = x[1]

    return 4*x1**2 + 10*x2**2 + 12*(x1**2)*(x2**2)

def f_for_plotting(x1: float, x2: float) -> float:
    """
    Same function defined above, used only for plotting
    """

    return 4*x1**2 + 10*x2**2 + 12*(x1**2)*(x2**2)

In [ ]:
plotting(f_for_plotting,'The objective function in [-5,5]x[-5,5]')

__Implement your own method__

If we do not know any optimization library, we have to implement our own algorithm to solve this optimization problem.

We are tackling a quite trivial problem. Let's also use the most basic unconstrained optimization method, the batch __gradient descent method__ with a fixed step-size $\eta$

The update rule is very simple. We fix $x_{k+1} = x_k - \eta \nabla f(x_k)$ until we get $$ \vert \vert \nabla f(x_k) \vert \vert \le \varepsilon $$

In [ ]:
# WE NEED TO IMPLEMENT THE GRADIENT FIRST
def df_dx(x: np.ndarray) -> np.array:
    """
    Computes gradient of function w.r.t. x variables
    NOTE: Return an array of gradients that has the same shape as the provided x!
    """

    x1 = x[0]
    x2 = x[1]

    df_dx1 = 8*x1 + 24*(x2**2)*x1
    df_dx2 = 20*x2 + 24*(x1**2)*x2

    return np.array([df_dx1, df_dx2])

In [ ]:
from typing import Callable, Dict, Any

def gradient_descent(
        f: Callable[[np.ndarray], np.ndarray],
        grad: Callable[[np.ndarray], np.ndarray],
        x_0: np.ndarray,
        eta: float,
        max_it: int,
        tol: float
) -> Dict[str, Any]:
    """
    Gradient descent algorithm.

    Parameters:
    f: function to to optimize
    grad: gradient of f
    x_0: starting point
    eta: step size,
    max_it: max iterations
    tol: tolerance (if norm of gradient < tol --> stationary point found)

    Returns:
    Dict {
        f_star: np.ndarray (best value)
        x_star: np.ndarray (best variables)
        grad_norm: np.ndarray (norm of gradient evaluated in x*)
        success: bool (True if stationary point found)
        iterations: int (number of iterations done)
    }
    """

    # INITIALIZATION
    x = x_0
    k = 0
    success = False
    
    for k in range(max_it):
        # COMPUTE GRADIENT
        df_dx = grad(x)

        # CHECK IF CURRENT POINT IS STATIONARY
        norm = np.linalg.norm(df_dx)
        if norm < tol:
            success = True
            break

        # CHECK IF GRADIENT EXPLODED
        if norm > 1e10:
            break

        # UPDATE CURRENT POINT
        x = x - eta * df_dx

    return {
        "f_star": f(x),
        "x_star": x,
        "grad_norm": grad(x),
        "success": success,
        "iterations": k
    }

In [ ]:
from pprint import pprint
import time

x_0 = np.array([-2,2])

start = time.time()
result = gradient_descent(f=f, grad=df_dx, x_0=x_0, eta=1e-4, max_it=100000, tol=1e-6)
end = time.time()

print("Result:")
pprint(result)

print("Duration: ", end - start)

And what if we do not want write down the gradient?

You can use __authomatic differentiation__ with __autograd__ library... but you will pay a __high price__

In [ ]:
from autograd import grad

x_0 = np.array([-0.5,0.5])

start = time.time()
result =  gradient_descent(f=f, grad=grad(f), x_0=x_0, eta=1e-4, max_it=100000, tol=1e-6)
end = time.time()

print("Result:")
pprint(result)

print("Duration: ", end - start)

__Almost 100 TIMES SLOWER!__

*Keep it in mind when deciding how to implement an optimization problem in Machine Learning*

What happens if we modify $\alpha$?

Rembember that the GD method has __sublinear convergence rate depending on $\alpha$__, meaning that:
1) The bigger $\alpha$, the faster we have convergence... __BUT__
2) If $\alpha$ is too big, the gradient explodes!!!

##### __TRY IT YOURSELF__
Change step size (eta) and observe what happens!

## SciPy

What if we have a more complex function?

What if we want to choose a more complex algorithm, such as BFGS or Conjugate Gradient?

Here we have __Scipy__

Documentation: https://docs.scipy.org/doc/scipy/reference/optimize.html

In [ ]:
from scipy.optimize import minimize

start = time.time()

x0 = np.random.random((2,))

res = minimize(
    fun=f, # OBJECTIVE FUNCTION TO OPTIMIZE
    x0=x0, # STARTING POINT
    method='BFGS', # OPTIMIZER
    jac=df_dx # GRADIENT FUNCTION
)

#NOTE that if we do not specify the jac argument, the gradient will be estimated automatically but it will be less precise
print(res.message)

print(f"Time for optimizing: {time.time()-start} seconds")
print(f"Solution {res.x}, Objective function {f(res.x)}, Gradient value {np.linalg.norm(df_dx(res.x))}")


Let's finally see what happens if we want to optimize a __function that depends also on parameters__.

This case is very frequent in Machine Learning, for example when you are optimizing an error function that depends also on the __hyper-parameters__, such as the regularization term.

Imagine you want to optimize
$$ f(x) = a_1 x_1^2 + a_2 x_2^2
$$

Where $x \in \mathbb{R}^2$ is the argument of the function, while $a_1,a_2 \in \mathbb{R}$ are two given parameters.

In this case we can write:

In [ ]:
def f(X, function_args) -> np.ndarray:  # X is a vector. X MUST BE the FIRST ARGUMENT!!!
    a1 = function_args[0] # unwrapping the arguments
    a2 = function_args[1]
    x1 = X[0] # unwrapping the variable
    x2 = X[1]
    return a1*x1**2 + a2*x2**2

def grad_f(X, function_args) -> np.ndarray:
    a1 = function_args[0] # unwrapping the arguments
    a2 = function_args[1]
    x1 = X[0] # unwrapping the variable
    x2 = X[1]
    d1 = 2*a1*x1
    d2 = 2*a2*x2
    return np.array([d1,d2])

#NOTE: function and graidient must take the same arguments as input!

#We define a starting point and two values for a1 and a2
x0 = np.array([1,3])

a1 = 1
a2 = 2

function_args = [a1,a2]

In [ ]:
import time

start=time.time()

res = minimize(f, x0, args=function_args,jac=grad_f,method='bfgs')

print(res.message)
print(f"Time for optimizing: {time.time()-start} seconds")
print(f"Solution {res.x}, Objective function {f(res.x, function_args)}, Gradient value {np.linalg.norm(grad_f(res.x, function_args))}")